# Exercises for Lecture 11 (Introduction to TensorFlow)

In [2]:
import datetime
now = datetime.datetime.now()
print("Last executed: " + now.strftime("%Y-%m-%d %H:%M:%S"))

Last executed: 2024-02-28 15:41:38


In [3]:
import numpy as np
import tensorflow as tf

## Exercise 1: Linear regression

### Load housing data

Load the California housing regression dataset.

In [4]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
(m, n)

(20640, 8)

In [5]:
housing.target.shape

(20640,)

In [6]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

The dataset contains 8 features (displayed above) contained in `housing.data` that can be used to predict house prices (in units of $100,000), which are contained in `housing.targets`.

### Set up data (add bias and reshape data)

In [7]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
housing_data_target = housing.target.reshape(-1, 1)

In [8]:
housing_data_target.shape, housing_data_plus_bias.shape

((20640, 1), (20640, 9))

Fit a linear regression model to the data, using a Scikit Learn LinearRegression model and then by computing the analytic solution using numpy and TensorFlow.  

Compare the fitted model between the different computations to ensure you get similar model parameters.

### Exercise 1a: Fit a linear regression model using Scikit Learn

Use the Scikit Learn LinearRegression model.

In [9]:
from sklearn.linear_model import LinearRegression

scikit_lin = LinearRegression().fit(housing_data_plus_bias, housing_data_target)
print(np.r_[scikit_lin.intercept_.reshape(-1, 1), scikit_lin.coef_.T])

[[-3.69419202e+01]
 [ 0.00000000e+00]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


### Exercise 1b: Fit a linear regression model using numpy

Solve the normal equations analytically.

In [10]:
theta = np.linalg.inv(housing_data_plus_bias.T.dot(housing_data_plus_bias)).dot(housing_data_plus_bias.T).dot(housing_data_target)
theta

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

### Exercise 1c: Fit a linear regression model using TensorFlow

Solve the normal equations analytically.

In [11]:
X = tf.constant(housing_data_plus_bias, name='X')
y = tf.constant(housing_data_target, name='y')

theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(tf.transpose(X), X)), tf.transpose(X)), y)
print(theta)

tf.Tensor(
[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]], shape=(9, 1), dtype=float64)


## Exercise 2: Compute gradients

Compute the partial derivatives of the following function (`my_func`) at `(a,b) = (0.2,0.3)` by:
1. numerical integration 
2. Autodiff in TensorFlow 

Check you get the same answer in both cases.

In [17]:
def my_func(a, b):
    z = 0.0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

### Compute by numerical integration

In [13]:
delta = 0.01
d_a = (my_func(0.2+delta, 0.3) - my_func(0.2-delta, 0.3)) / (2 * delta)
d_b = (my_func(0.2, 0.3+delta) - my_func(0.2, 0.3-delta)) / (2 * delta)

print(d_a, d_b)

-1.1383901861704486 0.19675140591134815


### Compute using Autodiff in TensorFlow

In [20]:
aa, bb = tf.Variable(0.2), tf.Variable(0.3)
with tf.GradientTape() as tape:
    z = 0.0
    for i in range(100):
        z = aa * tf.cos(z + i) + z * tf.sin(bb - i)

d_aabb = tape.gradient(z, [aa, bb])
print(d_aabb)

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.1388494>, <tf.Tensor: shape=(), dtype=float32, numpy=0.19671395>]
